In [1]:
# Cell 1: Install tenseal
!pip install tenseal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 44.6 MB/s eta 0:00:00


In [2]:
# Cell 2: Import libraries and load data
from google.colab import files
import json
import numpy as np
import tenseal as ts

# Upload the JSON file
uploaded = files.upload()
filename = list(uploaded.keys())[0]

with open(filename, "r") as f:
    data = json.load(f)

Saving MMU (1).json to MMU (1).json


In [3]:
# Cell 3: Inspect data
print("Users in original data:", list(data.keys())[:10])
print("Total users in data:", len(data))

if '1' not in data:
    print("WARNING: User '1' does not exist in the original data!")
    first_user_id = list(data.keys())[0]
    print(f"Using first available user: {first_user_id}")
else:
    first_user_id = '1'

Users in original data: ['1', '2', '3', '5', '6', '7', '8', '9', '10', '11']
Total users in data: 43


In [4]:
# Cell 4: Build encrypted database
# Secure storage for private keys
user_private_keys = {}

# Database to store encrypted data
encrypted_database = {}

print("\n=== BUILDING ENCRYPTED DATABASE ===")

for user_id, user_data in data.items():
    print(f"Processing User {user_id}...")

    try:
        # Generate encryption context for this user
        user_context = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=8192,
            coeff_mod_bit_sizes=[60, 40, 40, 60]
        )
        user_context.generate_galois_keys()
        user_context.global_scale = 2 ** 40

        # Serialize context WITH secret key
        user_private_key = user_context.serialize(save_secret_key=True)
        user_private_keys[user_id] = user_private_key
        print(f"  ✓ Private key stored for user {user_id}")

        # Create public context for server
        user_public_context = user_context.copy()
        user_public_context.make_context_public()

        # Encrypt feature vectors
        user_encrypted_features = []
        for vec in user_data["features"]:
            enc_vec = ts.ckks_vector(user_public_context, vec)
            user_encrypted_features.append(enc_vec.serialize())

        # Serialize public key
        user_public_key = user_public_context.serialize()

        # Store encrypted info
        encrypted_database[user_id] = {
            'public_key': user_public_key,
            'encrypted_features': user_encrypted_features
        }

        print(f"  ✓ Added User {user_id} to database")

    except Exception as e:
        print(f"  ✗ Error with User {user_id}: {e}")
        continue

print(f"✓ Enrollment completed. Stored private keys for {len(user_private_keys)} users")


=== BUILDING ENCRYPTED DATABASE ===
Processing User 1...
  ✓ Private key stored for user 1
  ✓ Added User 1 to database
Processing User 2...
  ✓ Private key stored for user 2
  ✓ Added User 2 to database
Processing User 3...
  ✓ Private key stored for user 3
  ✓ Added User 3 to database
Processing User 5...
  ✓ Private key stored for user 5
  ✓ Added User 5 to database
Processing User 6...
  ✓ Private key stored for user 6
  ✓ Added User 6 to database
Processing User 7...
  ✓ Private key stored for user 7
  ✓ Added User 7 to database
Processing User 8...
  ✓ Private key stored for user 8
  ✓ Added User 8 to database
Processing User 9...
  ✓ Private key stored for user 9
  ✓ Added User 9 to database
Processing User 10...
  ✓ Private key stored for user 10
  ✓ Added User 10 to database
Processing User 11...
  ✓ Private key stored for user 11
  ✓ Added User 11 to database
Processing User 12...
  ✓ Private key stored for user 12
  ✓ Added User 12 to database
Processing User 13...
  ✓ Priv

In [5]:
# Cell 5: Verify database
print("\n=== DATABASE VERIFICATION ===")
print("Users in encrypted database:", sorted(list(encrypted_database.keys()))[:10])
print(f"Total users in encrypted database: {len(encrypted_database)}")

for user_id in ['1', '2', '3', '44']:
    if user_id in encrypted_database:
        print(f"✓ User {user_id} found in database")
    else:
        print(f"✗ User {user_id} NOT found in database")


=== DATABASE VERIFICATION ===
Users in encrypted database: ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18']
Total users in encrypted database: 43
✓ User 1 found in database
✓ User 2 found in database
✓ User 3 found in database
✓ User 44 found in database


In [6]:
# Cell 6: Cosine similarity function
def cosine_similarity_encrypted_public(v1, v2):
    """
    Compute cosine similarity using only homomorphic operations.
    Returns encrypted similarity score.
    """
    # Encrypted dot product
    dot_product = v1.dot(v2)

    # Encrypted squared norms
    norm_sq_v1 = v1.dot(v1)
    norm_sq_v2 = v2.dot(v2)

    # Return encrypted components
    return dot_product, norm_sq_v1, norm_sq_v2

In [7]:
# Cell 7: XAI Class Definition
class AuthenticationXAI:
    def __init__(self, threshold=0.70):
        self.threshold = threshold

    def explain_authentication(self, query_user_id, template_scores, feature_similarities_all, threshold=0.70):
        """
        Generate human-readable explanations for authentication decisions.
        """

        print(f"\n{'='*60}")
        print("EXPLAINABLE AI (XAI) - AUTHENTICATION DECISION ANALYSIS")
        print(f"{'='*60}")

        if not template_scores:
            print("ERROR: No template scores available for analysis")
            return

        best_template_idx = np.argmax(template_scores)
        best_score = template_scores[best_template_idx]

        is_authenticated = best_score >= threshold

        print(f"\n🔍 USER ANALYSIS: {query_user_id}")
        print(f"📊 BEST MATCH SCORE: {best_score:.4f}")
        print(f"🎯 THRESHOLD: {threshold}")
        print(f"📋 AUTHENTICATION: {'SUCCESSFUL ✅' if is_authenticated else 'FAILED ❌'}")
        print(f"📈 MARGIN: {best_score - threshold:+.4f}")

        # Overall score analysis
        print(f"\n📊 OVERALL SCORE ANALYSIS:")
        print(f"   • Templates analyzed: {len(template_scores)}")
        print(f"   • Templates above threshold: {sum(1 for s in template_scores if s >= threshold)}")
        print(f"   • Average template score: {np.mean(template_scores):.4f}")
        print(f"   • Score variance: {np.var(template_scores):.4f}")

        if len(template_scores) > 1:
            sorted_scores = sorted(template_scores, reverse=True)
            if sorted_scores[0] - sorted_scores[1] > 0.1:
                print(f"   • Best template significantly outperforms others")
            elif sorted_scores[0] - sorted_scores[1] < 0.05:
                print(f"   • Multiple templates show similar performance")

        # Feature-level analysis if available
        if feature_similarities_all and len(feature_similarities_all) > best_template_idx:
            feature_scores = feature_similarities_all[best_template_idx]
            if feature_scores:
                print(f"\n📈 FEATURE-LEVEL ANALYSIS (Best Template {best_template_idx + 1}):")

                for i, score in enumerate(feature_scores):
                    status = "✓" if score >= 0.7 else ("∼" if score >= 0.5 else "✗")
                    contribution = "HIGH" if score >= 0.8 else ("MEDIUM" if score >= 0.6 else "LOW")
                    print(f"   Feature {i+1}: {status} {score:.4f} ({contribution})")

                # Identify strongest and weakest features
                if len(feature_scores) > 1:
                    best_feature = np.argmax(feature_scores)
                    worst_feature = np.argmin(feature_scores)

                    if feature_scores[best_feature] >= 0.8:
                        print(f"   → Feature {best_feature + 1} was strongest contributor")
                    if feature_scores[worst_feature] < 0.5:
                        print(f"   → Feature {worst_feature + 1} was weakest contributor")

        # Decision explanation
        print(f"\n🎯 DECISION EXPLANATION:")
        if is_authenticated:
            print(f"   1. Template {best_template_idx + 1} achieved {best_score*100:.1f}% similarity")
            print(f"   2. Exceeded threshold by {(best_score - threshold)*100:.1f}%")

            if best_score >= 0.85:
                print(f"   3. Very strong biometric match detected")
            elif best_score >= 0.75:
                print(f"   3. Strong biometric match confirmed")
            else:
                print(f"   3. Adequate biometric match established")

            # Check consistency
            high_scoring_templates = sum(1 for s in template_scores if s >= threshold)
            if high_scoring_templates >= 3:
                print(f"   4. Multiple templates ({high_scoring_templates}) support authentication")

        else:
            print(f"   1. Best template only achieved {best_score*100:.1f}% similarity")
            print(f"   2. Fell short of threshold by {(threshold - best_score)*100:.1f}%")

            if best_score >= 0.65:
                print(f"   3. Close match but below security threshold")
                print(f"   4. Consider using more features or adjusting threshold slightly")
            elif best_score >= 0.5:
                print(f"   3. Moderate match - possible biometric variation")
                print(f"   4. Recommend re-capturing biometric data")
            else:
                print(f"   3. Poor match - likely incorrect user or data issue")
                print(f"   4. Consider re-enrollment")

        # Confidence assessment
        print(f"\n💡 CONFIDENCE ASSESSMENT:")
        if best_score >= 0.85:
            confidence = "VERY HIGH"
        elif best_score >= 0.75:
            confidence = "HIGH"
        elif best_score >= threshold:
            confidence = "MODERATE"
        elif best_score >= 0.6:
            confidence = "LOW"
        else:
            confidence = "VERY LOW"

        print(f"   • Decision Confidence: {confidence}")

        if is_authenticated:
            if confidence in ["VERY HIGH", "HIGH"]:
                print(f"   • Recommended Action: Grant full access")
            else:
                print(f"   • Recommended Action: Grant limited access + additional verification")
        else:
            if best_score >= 0.65:
                print(f"   • Recommended Action: Request re-authentication with more features")
            else:
                print(f"   • Recommended Action: Deny access + security alert")

        print(f"\n{'='*60}")
        print("XAI ANALYSIS COMPLETE")
        print(f"{'='*60}")

In [8]:
# Cell 8: Authentication Setup
query_user_id = '40'
print(f"\n=== AUTHENTICATING USER {query_user_id} ===")

if query_user_id not in encrypted_database:
    print(f"ERROR: User {query_user_id} not found!")
    exit()

# Get query vectors
query_features = [
    data[query_user_id]["features"][1],  # Feature 1
    data[query_user_id]["features"][2]   # Feature 2
]
print(f"✓ Selected 2 query features: Feature 2 and Feature 3")

# Retrieve target user's public context
target_data = encrypted_database[query_user_id]
target_public_context = ts.context_from(target_data['public_key'])

# Encrypt query vectors
enc_queries = []
for i, query_vec in enumerate(query_features):
    enc_query = ts.ckks_vector(target_public_context, query_vec)
    enc_queries.append(enc_query)
print("✓ Both query features encrypted with correct public key")

# Retrieve private context
try:
    if query_user_id not in user_private_keys:
        print(f"✗ No private key found for user {query_user_id}")
        exit()

    user_private_context = ts.context_from(user_private_keys[query_user_id])
    print("✓ Retrieved private context with secret key")

except Exception as e:
    print(f"✗ Failed to retrieve private context: {e}")
    exit()


=== AUTHENTICATING USER 40 ===
✓ Selected 2 query features: Feature 2 and Feature 3
✓ Both query features encrypted with correct public key
✓ Retrieved private context with secret key


In [9]:
# Cell 9: Server-side Comparison
print(f"\n=== SERVER-SIDE COMPARISON (Processing {len(enc_queries)} features) ===")

target_data = encrypted_database[query_user_id]
target_public_context = ts.context_from(target_data['public_key'])
target_enc_features = target_data['encrypted_features']

encrypted_results_all = []  # This will be a 2D list: [template_index][feature_index]

for template_idx, enc_feat_serialized in enumerate(target_enc_features):
    try:
        # Deserialize encrypted template vector
        enc_feat = ts.ckks_vector_from(target_public_context, enc_feat_serialized)

        template_results = []

        # Compare each query feature with this template
        for feature_idx, enc_query in enumerate(enc_queries):
            # Compute encrypted similarity
            dot_product, norm_sq_v1, norm_sq_v2 = cosine_similarity_encrypted_public(enc_query, enc_feat)

            # Store encrypted results
            template_results.append({
                'dot_product': dot_product.serialize(),
                'norm_sq_v1': norm_sq_v1.serialize(),
                'norm_sq_v2': norm_sq_v2.serialize()
            })

        # Add all feature results
        encrypted_results_all.append(template_results)
        print(f"  ✓ Template {template_idx + 1}: All {len(enc_queries)} feature comparisons completed")

    except Exception as e:
        print(f"  ✗ Template {template_idx + 1}: Error during comparison: {e}")
        encrypted_results_all.append([])


=== SERVER-SIDE COMPARISON (Processing 2 features) ===
  ✓ Template 1: All 2 feature comparisons completed
  ✓ Template 2: All 2 feature comparisons completed
  ✓ Template 3: All 2 feature comparisons completed
  ✓ Template 4: All 2 feature comparisons completed
  ✓ Template 5: All 2 feature comparisons completed
  ✓ Template 6: All 2 feature comparisons completed
  ✓ Template 7: All 2 feature comparisons completed
  ✓ Template 8: All 2 feature comparisons completed
  ✓ Template 9: All 2 feature comparisons completed
  ✓ Template 10: All 2 feature comparisons completed


In [11]:
# Cell 10: Client-side Decryption with XAI Data Collection
print(f"\n=== CLIENT-SIDE DECRYPTION AND AVERAGE COMPUTATION ===")

decrypted_average_scores = []
feature_similarities_all = []  # NEW: Store feature-level scores for XAI
successful_decryptions = 0

for template_idx, template_results in enumerate(encrypted_results_all):
    if not template_results:  # Skip failed comparisons
        print(f"  ✗ Template {template_idx + 1}: Skipping - no comparison results")
        decrypted_average_scores.append(0.0)
        feature_similarities_all.append([])  # Empty list for this template
        continue

    try:
        feature_similarities = []

        # Decrypt and compute similarity for each feature
        for feature_idx, result in enumerate(template_results):
            # Deserialize encrypted vectors with private context
            dot_product_enc = ts.ckks_vector_from(user_private_context, result['dot_product'])
            norm_sq_v1_enc = ts.ckks_vector_from(user_private_context, result['norm_sq_v1'])
            norm_sq_v2_enc = ts.ckks_vector_from(user_private_context, result['norm_sq_v2'])

            # Decrypt the components
            dot_product = dot_product_enc.decrypt()[0]
            norm_sq_v1_val = norm_sq_v1_enc.decrypt()[0]
            norm_sq_v2_val = norm_sq_v2_enc.decrypt()[0]

            # Calculate cosine similarity safely
            norm_sq_v1_val = max(0, norm_sq_v1_val)
            norm_sq_v2_val = max(0, norm_sq_v2_val)
            norm_v1 = np.sqrt(norm_sq_v1_val) if norm_sq_v1_val > 0 else 0
            norm_v2 = np.sqrt(norm_sq_v2_val) if norm_sq_v2_val > 0 else 0

            if norm_v1 == 0 or norm_v2 == 0:
                similarity = 0.0
            else:
                similarity = dot_product / (norm_v1 * norm_v2)

            similarity = max(-1.0, min(1.0, similarity))
            feature_similarities.append(similarity)

        # Store feature-level scores for XAI
        feature_similarities_all.append(feature_similarities)

        # Calculate average similarity
        average_similarity = sum(feature_similarities) / len(feature_similarities)
        decrypted_average_scores.append(average_similarity)
        successful_decryptions += 1

        # Display results
        print(f"  ✓ Template {template_idx + 1}:")
        for i, sim in enumerate(feature_similarities):
            print(f"      Feature {i + 1} similarity: {sim:.4f}")
        print(f"      Average similarity: {average_similarity:.4f}")

    except Exception as e:
        print(f"  ✗ Template {template_idx + 1}: Error decrypting: {e}")
        decrypted_average_scores.append(0.0)
        feature_similarities_all.append([])


=== CLIENT-SIDE DECRYPTION AND AVERAGE COMPUTATION ===
  ✓ Template 1:
      Feature 1 similarity: -0.1998
      Feature 2 similarity: -0.0683
      Average similarity: -0.1340
  ✓ Template 2:
      Feature 1 similarity: 1.0000
      Feature 2 similarity: 0.7191
      Average similarity: 0.8596
  ✓ Template 3:
      Feature 1 similarity: 0.7191
      Feature 2 similarity: 1.0000
      Average similarity: 0.8596
  ✓ Template 4:
      Feature 1 similarity: 0.9886
      Feature 2 similarity: 0.6841
      Average similarity: 0.8363
  ✓ Template 5:
      Feature 1 similarity: -0.0483
      Feature 2 similarity: 0.6082
      Average similarity: 0.2799
  ✓ Template 6:
      Feature 1 similarity: 0.4603
      Feature 2 similarity: -0.1879
      Average similarity: 0.1362
  ✓ Template 7:
      Feature 1 similarity: -0.5925
      Feature 2 similarity: -0.8204
      Average similarity: -0.7065
  ✓ Template 8:
      Feature 1 similarity: -0.2519
      Feature 2 similarity: -0.6472
      Average s

In [12]:
# Cell 11: Authentication Results
print(f"\n=== FINAL AUTHENTICATION RESULTS (Average of {len(query_features)} Features) ===")
print(f"Successful decryptions: {successful_decryptions}/{len(target_enc_features)}")

if successful_decryptions > 0:
    # Create list with (user_id, average_similarity) tuples
    client_side_scores = [(query_user_id, score) for score in decrypted_average_scores]
    client_side_scores = sorted(client_side_scores, key=lambda x: x[1], reverse=True)

    print(f"\nTop Matches (Average of {len(query_features)} Features):")
    for i, (uid, avg_sim) in enumerate(client_side_scores[:5]):
        print(f"  {i+1}. User {uid} [Template {i+1}] -> Average Similarity: {avg_sim:.4f}")

    threshold = 0.70
    best_match = client_side_scores[0]

    print(f"\nAuthentication Threshold: {threshold}")
    print(f"Best Match Average Similarity: {best_match[1]:.4f}")

    if best_match[1] >= threshold:
        print(f"✓ Authenticated as User {best_match[0]} with average similarity {best_match[1]:.4f}")
    else:
        print(f"✗ Authentication failed. Best match is User {best_match[0]} with average similarity {best_match[1]:.4f}")

    # Additional diagnostic information
    print(f"\n=== DIAGNOSTIC INFORMATION ===")
    print(f"Number of query features used: {len(query_features)}")
    print(f"Number of templates processed: {len(target_enc_features)}")

    # Show individual template results
    if decrypted_average_scores:
        print(f"\nIndividual Template Results:")
        for i, avg_score in enumerate(decrypted_average_scores):
            if avg_score > 0:
                print(f"  Template {i+1}: Average Score = {avg_score:.4f}")

else:
    print("✗ No successful decryptions - authentication failed")


=== FINAL AUTHENTICATION RESULTS (Average of 2 Features) ===
Successful decryptions: 10/10

Top Matches (Average of 2 Features):
  1. User 40 [Template 1] -> Average Similarity: 0.8596
  2. User 40 [Template 2] -> Average Similarity: 0.8596
  3. User 40 [Template 3] -> Average Similarity: 0.8363
  4. User 40 [Template 4] -> Average Similarity: 0.5781
  5. User 40 [Template 5] -> Average Similarity: 0.3940

Authentication Threshold: 0.7
Best Match Average Similarity: 0.8596
✓ Authenticated as User 40 with average similarity 0.8596

=== DIAGNOSTIC INFORMATION ===
Number of query features used: 2
Number of templates processed: 10

Individual Template Results:
  Template 2: Average Score = 0.8596
  Template 3: Average Score = 0.8596
  Template 4: Average Score = 0.8363
  Template 5: Average Score = 0.2799
  Template 6: Average Score = 0.1362
  Template 9: Average Score = 0.5781
  Template 10: Average Score = 0.3940


In [13]:
# Cell 12: XAI Analysis
print(f"\n{'='*60}")
print("XAI DECISION EXPLANATION")
print(f"{'='*60}")

# Simple XAI explanation
best_score = max(decrypted_average_scores) if decrypted_average_scores else 0
threshold = 0.70

print(f"\n🔍 QUICK ANALYSIS:")
print(f"   User: {query_user_id}")
print(f"   Best Score: {best_score:.4f}")
print(f"   Threshold: {threshold}")
print(f"   Result: {'AUTHENTICATED ✅' if best_score >= threshold else 'REJECTED ❌'}")

if best_score >= threshold:
    print(f"\n✅ REASONS FOR AUTHENTICATION:")
    print(f"   1. Strong match: {best_score*100:.1f}% similarity")
    print(f"   2. Exceeds threshold by {(best_score - threshold)*100:.1f}%")

    successful_templates = sum(1 for s in decrypted_average_scores if s >= threshold)
    print(f"   3. {successful_templates}/{len(decrypted_average_scores)} templates confirm identity")

    if successful_templates >= 3:
        print(f"   4. High consistency across multiple templates")

    # Check feature performance
    if feature_similarities_all:
        best_template_idx = np.argmax(decrypted_average_scores)
        if best_template_idx < len(feature_similarities_all):
            best_features = feature_similarities_all[best_template_idx]
            strong_features = sum(1 for f in best_features if f >= 0.7)
            if strong_features > 0:
                print(f"   5. {strong_features}/{len(best_features)} features showed strong matches")

    print(f"\n💡 CONFIDENCE: {'HIGH' if best_score > 0.8 else 'MODERATE'}")

else:
    print(f"\n❌ REASONS FOR REJECTION:")
    print(f"   1. Insufficient match: {best_score*100:.1f}% similarity")
    print(f"   2. Below threshold by {(threshold - best_score)*100:.1f}%")

    close_templates = [s for s in decrypted_average_scores if 0.65 <= s < threshold]
    if close_templates:
        print(f"   3. {len(close_templates)} templates were close to threshold")
        print(f"   4. Consider: Adjusting threshold or adding more features")
    else:
        print(f"   3. Poor match across all templates")
        print(f"   4. Consider: Re-enrollment or verifying user identity")

    print(f"\n⚠️  RECOMMENDATION: {'Re-authenticate with more features' if best_score >= 0.6 else 'Security review recommended'}")

# Run detailed XAI analysis
print(f"\n{'='*60}")
print("DETAILED XAI ANALYSIS")
print(f"{'='*60}")

xai_analyzer = AuthenticationXAI(threshold=0.70)
xai_analyzer.explain_authentication(
    query_user_id=query_user_id,
    template_scores=decrypted_average_scores,
    feature_similarities_all=feature_similarities_all,
    threshold=0.70
)


XAI DECISION EXPLANATION

🔍 QUICK ANALYSIS:
   User: 40
   Best Score: 0.8596
   Threshold: 0.7
   Result: AUTHENTICATED ✅

✅ REASONS FOR AUTHENTICATION:
   1. Strong match: 86.0% similarity
   2. Exceeds threshold by 16.0%
   3. 3/10 templates confirm identity
   4. High consistency across multiple templates
   5. 2/2 features showed strong matches

💡 CONFIDENCE: HIGH

DETAILED XAI ANALYSIS

EXPLAINABLE AI (XAI) - AUTHENTICATION DECISION ANALYSIS

🔍 USER ANALYSIS: 40
📊 BEST MATCH SCORE: 0.8596
🎯 THRESHOLD: 0.7
📋 AUTHENTICATION: SUCCESSFUL ✅
📈 MARGIN: +0.1596

📊 OVERALL SCORE ANALYSIS:
   • Templates analyzed: 10
   • Templates above threshold: 3
   • Average template score: 0.2654
   • Score variance: 0.2779
   • Multiple templates show similar performance

📈 FEATURE-LEVEL ANALYSIS (Best Template 2):
   Feature 1: ✓ 1.0000 (HIGH)
   Feature 2: ✓ 0.7191 (MEDIUM)
   → Feature 1 was strongest contributor

🎯 DECISION EXPLANATION:
   1. Template 2 achieved 86.0% similarity
   2. Exceeded 